# CNN Model 
## Pneumonia Classification from Chest X-rays 

## Data Exploration

In [32]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Define the data directories
train_dir = 'C:/Users/hasan/Desktop/University of Liverpool/7- Deep Learning CSCK506  Jan 2023 A/Week 8/archive(1)/chest_xray/train'
test_dir = 'C:/Users/hasan/Desktop/University of Liverpool/7- Deep Learning CSCK506  Jan 2023 A/Week 8/archive(1)/chest_xray/test'
val_dir = 'C:/Users/hasan/Desktop/University of Liverpool/7- Deep Learning CSCK506  Jan 2023 A/Week 8/archive(1)/chest_xray/val'

In [3]:
# Define the data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),  # Resize the input images to 150x150 pixels
        batch_size=32,
        class_mode='binary')     # The problem is binary classification, so we use binary labels

Found 5216 images belonging to 2 classes.


In [5]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 624 images belonging to 2 classes.


In [6]:
val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 16 images belonging to 2 classes.


In [30]:
train_normal = os.path.join(train_dir, 'NORMAL')
train_pneumonia = os.path.join(train_dir, 'PNEUMONIA')
test_normal = os.path.join(test_dir, 'NORMAL')
test_pneumonia = os.path.join(test_dir, 'PNEUMONIA')
val_normal = os.path.join(val_dir, 'NORMAL')
val_pneumonia = os.path.join(val_dir, 'PNEUMONIA')

In [33]:
# Print sizes of some images

for file in os.listdir(train_normal)[:10]:
  img = load_img(os.path.join(train_normal, file))
  print(img.size)
for file in os.listdir(test_pneumonia)[:10]:
  img = load_img(os.path.join(test_pneumonia, file))
  print(img.size)

(2090, 1858)
(1422, 1152)
(1810, 1434)
(1618, 1279)
(1600, 1125)
(1974, 1306)
(1528, 1013)
(1384, 1167)
(1450, 1144)
(1468, 993)
(1106, 762)
(1024, 680)
(1128, 624)
(904, 552)
(880, 536)
(952, 624)
(1040, 696)
(1064, 720)
(1064, 840)
(1032, 752)


## Model 1
### CNN

In [10]:
# Define the CNN architecture
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),  # First convolutional layer with 150 filters and a 3x3 kernel
    layers.MaxPooling2D((2, 2)),  # Max pooling layer with a 2x2 pool size
    
    layers.Conv2D(64, (3, 3), activation='relu'),  # Second convolutional layer with 64 filters and a 3x3 kernel
    layers.MaxPooling2D((2, 2)),  # Max pooling layer with a 2x2 pool size
    
    layers.Conv2D(128, (3, 3), activation='relu'),  # Third convolutional layer with 128 filters and a 3x3 kernel
    layers.MaxPooling2D((2, 2)),  # Max pooling layer with a 2x2 pool size
    
    layers.Flatten(),  # Flatten the output from the convolutional layers to a 1D vector
    layers.Dense(128, activation='relu'),  # Fully connected layer with 128 neurons
    layers.Dense(1, activation='sigmoid')  # Output layer with a single neuron and a sigmoid activation function
])

In [11]:
# Compile the model
model.compile(optimizer='adam',  # Use the Adam optimizer
              loss='binary_crossentropy',  # Use binary cross-entropy loss function
              metrics=['accuracy'])

In [12]:
# Train the model
model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples//train_generator.batch_size,
      epochs=10,
      validation_data=val_generator,
      validation_steps=val_generator.samples//val_generator.batch_size)

Epoch 1/10
163/163 [==============================] - 203s 1s/step - loss: 0.2095 - accuracy: 0.9143
Epoch 2/10
163/163 [==============================] - 192s 1s/step - loss: 0.0863 - accuracy: 0.9686
Epoch 3/10
163/163 [==============================] - 191s 1s/step - loss: 0.0772 - accuracy: 0.9691
Epoch 4/10
163/163 [==============================] - 190s 1s/step - loss: 0.0626 - accuracy: 0.9789
Epoch 5/10
163/163 [==============================] - 189s 1s/step - loss: 0.0487 - accuracy: 0.9839
Epoch 6/10
163/163 [==============================] - 190s 1s/step - loss: 0.0415 - accuracy: 0.9841
Epoch 7/10
163/163 [==============================] - 190s 1s/step - loss: 0.0266 - accuracy: 0.9893
Epoch 8/10
163/163 [==============================] - 189s 1s/step - loss: 0.0322 - accuracy: 0.9877
Epoch 9/10
163/163 [==============================] - 191s 1s/step - loss: 0.0328 - accuracy: 0.9873
Epoch 10/10
163/163 [==============================] - 192s 1s/step - loss: 0.0152 - accura

In [13]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, verbose=2)

20/20 - 17s - loss: 2.6868 - accuracy: 0.7324 - 17s/epoch - 830ms/step


In [14]:
# Print the test loss and accuracy scores
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Test loss: 2.68679141998291
Test accuracy: 0.7323718070983887


## Model 2
### Hyperparameters Added

In [60]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory('C:/Users/hasan/Desktop/University of Liverpool/7- Deep Learning CSCK506  Jan 2023 A/Week 8/archive(1)/chest_xray/train',
                                                    target_size=(256, 256),
                                                    batch_size=32,
                                                    class_mode='binary')

val_generator = test_datagen.flow_from_directory('C:/Users/hasan/Desktop/University of Liverpool/7- Deep Learning CSCK506  Jan 2023 A/Week 8/archive(1)/chest_xray/val',
                                                        target_size=(256, 256),
                                                        batch_size=32,
                                                        class_mode='binary')

test_generator = test_datagen.flow_from_directory('C:/Users/hasan/Desktop/University of Liverpool/7- Deep Learning CSCK506  Jan 2023 A/Week 8/archive(1)/chest_xray/test',
                                                        target_size=(256, 256), 
                                                        batch_size=32,
                                                        class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [61]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras import layers
import tensorflow as tf



def grid_model(padding='valid',
               strides=2,
               activation='relu',
               optimizer='adam',
               dropout=0.3):

    
    # Define the CNN model
    model = Sequential()
    
    model.add(Conv2D(32, (3,3), activation=activation, padding=padding, strides=strides, input_shape=(256, 256, 3)))
    model.add(MaxPooling2D((2,2)))
    #model.add(BatchNormalization())
    #model.add(Dropout(dropout))
    
    model.add(Conv2D(64, (3,3), activation=activation, padding=padding, strides=strides))
    model.add(MaxPooling2D((2,2)))
    #model.add(Dropout(dropout))
   
    model.add(Conv2D(128, (3,3), activation=activation, padding=padding, strides=strides))
    model.add(MaxPooling2D((2,2)))
    #model.add(Dropout(dropout))
    #model.add(BatchNormalization())
    
    model.add(Flatten())

    model.add(Dense(128, activation=activation))
    model.add(Dropout(dropout))
    #model.add(BatchNormalization())
    
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                optimizer=optimizer,
                metrics=['acc'])
    
    return model

In [62]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

estimator = KerasClassifier(build_fn=grid_model, epochs=10, batch_size=32, verbose=0)

C:\Users\hasan\AppData\Local\Temp/ipykernel_19496/1379658813.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=grid_model, epochs=10, batch_size=32, verbose=0)


In [63]:
# Define the hyperparameters to search over
param_grid = {
    'padding': ['valid', 'same'],
    'strides': [(1, 1), (2, 2)],
    'activation': ['relu', 'tanh'],
    'optimizer': ['adam', 'rmsprop']
}

In [64]:
from sklearn.model_selection import GridSearchCV

# Create a grid search object
grid_search = GridSearchCV(estimator, param_grid, cv=3, n_jobs=-1, scoring='accuracy')

In [65]:
grid_search.fit(val_X, val_y)

GridSearchCV(cv=3,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x0000022821C76FA0>,
             n_jobs=-1,
             param_grid={'activation': ['relu', 'tanh'],
                         'optimizer': ['adam', 'rmsprop'],
                         'padding': ['valid', 'same'],
                         'strides': [(1, 1), (2, 2)]},
             scoring='accuracy')

In [71]:
print("Best: %f using %s" % (grid_search.best_score_, grid_search.best_params_))
best_model = grid_search.best_estimator_

Best: 0.666667 using {'activation': 'relu', 'optimizer': 'adam', 'padding': 'same', 'strides': (1, 1)}


In [72]:
model = grid_model(padding='same', strides=(1,1),
           activation='relu', optimizer='adam')

In [79]:
#history = model.fit(train_X, train_y,
                      #steps_per_epoch=train_X.shape[0] // 12,
 #                   steps_per_epoch=train_generator.samples//train_generator.batch_size
  #                    batch_size=32,
   #                   epochs=10)

# Train the model
model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples//train_generator.batch_size,
      epochs=10,
      validation_data=val_generator,
      validation_steps=val_generator.samples//val_generator.batch_size)

Epoch 1/10
163/163 [==============================] - 487s 3s/step - loss: 0.3944 - acc: 0.8345
Epoch 2/10
163/163 [==============================] - 460s 3s/step - loss: 0.2144 - acc: 0.9153
Epoch 3/10
163/163 [==============================] - 478s 3s/step - loss: 0.1915 - acc: 0.9270
Epoch 4/10
163/163 [==============================] - 493s 3s/step - loss: 0.1681 - acc: 0.9375
Epoch 5/10
163/163 [==============================] - 508s 3s/step - loss: 0.1516 - acc: 0.9433
Epoch 6/10
163/163 [==============================] - 488s 3s/step - loss: 0.1525 - acc: 0.9423
Epoch 7/10
163/163 [==============================] - 484s 3s/step - loss: 0.1407 - acc: 0.9457
Epoch 8/10
163/163 [==============================] - 485s 3s/step - loss: 0.1417 - acc: 0.9459
Epoch 9/10
163/163 [==============================] - 488s 3s/step - loss: 0.1327 - acc: 0.9528
Epoch 10/10
163/163 [==============================] - 501s 3s/step - loss: 0.1196 - acc: 0.9567


In [82]:
test_loss, test_acc = model.evaluate(test_generator, verbose=2)

# Print the test loss and accuracy
print(f'Test loss: {test_loss:.4f}')
print(f'Test accuracy: {test_acc:.4f}')

20/20 - 21s - loss: 0.4997 - acc: 0.9054 - 21s/epoch - 1s/step
Test loss: 0.4997
Test accuracy: 0.9054
